In [1]:
import numpy as np
import pandas as pd
import time

# @param dfFile: pandas.DataFrame ('nba_preprocessed.csv')
# @param dateStart, dateEnd: str in the format of 'YYYY-MM-DD'
# @param attriToDrop: list[str]
# @return X, Y: pandas.DataFrame
# featureExtraction() outputs X, Y for model training.
# Game date can be assigned
# Attribute to be dropped can be assigned
def featureExtraction(dfFile, dateStart='1000-01-01', dateEnd='2999-12-31', attriToDrop=None):
    df = pd.read_csv(dfFile)
    
    # Date selection
    df = df.loc[lambda df: (df.Date_A > dateStart) & (df.Date_A < dateEnd), :].reset_index(drop=True)
    
    # Get label Y
    Y = df[['W/L_A']]
    Y = Y.rename(columns={'W/L_A': 'Label'})
    
    # Get attributes X
    colToDrop = ['Team_A', 'Date_A', 'W/L_A', 'Score_A', 'Opponent_A', 'Team_B', 'Date_B', 'W/L_B', 'Home/Away_B', 'Score_B', 'Opponent_B']
    colToDrop += attriToDrop if attriToDrop else []
    X = df.drop(columns = colToDrop)
    
    return X, Y

In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from matplotlib import pyplot as plt
from sklearn.svm import SVC
import numpy as np
import operator
%matplotlib inline

def CrossValidationGridSearchNested(X_data, Y_data, num_trials, fold_num, est_classifcation, tuned_param, scoring):
    max_score = -1
    best_estimator = est_classifcation
    
    for i in range(num_trials):
        inner_cv = StratifiedKFold(n_splits=fold_num, random_state=i, shuffle=True)
        outer_cv = StratifiedKFold(n_splits=fold_num, random_state=i+1, shuffle=True)

        # Non_nested parameter search and scoring
        clf = GridSearchCV(estimator=est_classifcation, param_grid=tuned_param, cv=inner_cv, scoring=scoring)
        clf.fit(X_data, Y_data)
        
        # CV with parameter optimization
        param_score = cross_val_score(clf.best_estimator_, X=X_data, y=Y_data, cv=outer_cv, scoring=scoring).mean()
        if(param_score > max_score):
            max_score = param_score
            best_estimator = clf.best_estimator_
            
        progress = (i+1)/num_trials*100
        print(f'> progress = {progress}%')
    
    return (max_score, best_estimator)

In [3]:
dfFile = '../NBA_Ino_part/nba_preprocessed.csv'
dateStart = '2017-10-01'
dateEnd = '2018-04-30'
# X, Y = featureExtraction(dfFile, dateStart, dateEnd)
X, Y = featureExtraction(dfFile, attriToDrop=['PTS_A', 'PTS_B'], dateStart=dateStart, dateEnd=dateEnd)
# X, Y = featureExtraction(dfFile)
X_val = X.values
Y_val = Y.values

In [4]:
import lightgbm as lgb  
from lightgbm import LGBMClassifier 
from time import time

params = {'boosting_type': 'gbdt',
          'max_depth' : -1,
          'objective': 'binary', 
          'nthread': 5, # Updated from nthread
          'num_leaves': 64, 
          'learning_rate': 0.05, 
          'max_bin': 512, 
          'subsample_for_bin': 200,
          'subsample': 1, 
          'subsample_freq': 1, 
          'colsample_bytree': 0.8, 
          'reg_alpha': 5, 
          'reg_lambda': 10,
          'min_split_gain': 0.5, 
          'min_child_weight': 1, 
          'min_child_samples': 5, 
          'scale_pos_weight': 1,
          'num_class' : 1,
          'metric' : 'binary_error',
          'cat_smooth' : 10,
          'objective' : 'binary',
          'metric' : 'auc'}

# Set the parameters by cross-validation
tuned_parameters = {
    'learning_rate': [0.1],
    'n_estimators': range(100, 1001, 400),
    "max_depth": [11, 12],
    #'num_leaves': range(30, 100, 30),
    'boosting_type' : ['gbdt'],
    #'random_state' : [501], # Updated from 'seed'
    'colsample_bytree' : [0.65],
    'subsample' : [0.5]
    #'reg_alpha' : [1, 1.2],
    #'reg_lambda' : [1, 1.2, 1.4],
    }

# Number of random trials
NUM_TRIALS = 1

# We will use a Support Vector Classifier with "rbf" kernel
#lgbm = LGBMClassifier()
lgbm = LGBMClassifier(boosting_type= 'gbdt',  
          n_jobs = -1, # Updated from 'nthread'
          silent = True,
          max_depth = params['max_depth'],
          max_bin = params['max_bin'], 
          subsample_for_bin = params['subsample_for_bin'],
          subsample = params['subsample'], 
          subsample_freq = params['subsample_freq'], 
          min_split_gain = params['min_split_gain'], 
          min_child_weight = params['min_child_weight'], 
          min_child_samples = params['min_child_samples'], 
          scale_pos_weight = params['scale_pos_weight'],
          cat_smooth = params['cat_smooth'],
          objective = params['objective'],
          reg_alpha = params['reg_alpha'],
          reg_lambda = params['reg_lambda'],
          metric = params['metric'])

start = time()
(max_score, lgbm_best_estimator) = CrossValidationGridSearchNested(X_val, Y_val.ravel(), NUM_TRIALS, 10, lgbm, tuned_parameters, 'roc_auc')
lgbm_best_parameter = lgbm_best_estimator.get_params()

print("CrossValidationGridSearchNested of LightGradientBoostingClassifier wih NUM_TRIALS = %2.0d took %.2f seconds."%(NUM_TRIALS, (time() - start)))
print(f'\nmax_score = {max_score}\n')
print(f'\nbest_estimator = {lgbm_best_estimator}\n')
print(f'\nbest_parameter = {lgbm_best_parameter}\n')

> progress = 100.0%
CrossValidationGridSearchNested of LightGradientBoostingClassifier wih NUM_TRIALS =  1 took 56815.07 seconds.

max_score = 0.9872893119175095


best_estimator = LGBMClassifier(boosting_type='gbdt', cat_smooth=10, class_weight=None,
        colsample_bytree=0.65, learning_rate=0.1, max_bin=512,
        max_depth=11, metric='auc', min_child_samples=5,
        min_child_weight=1, min_split_gain=0.5, n_estimators=500,
        n_jobs=-1, num_leaves=31, objective='binary', random_state=None,
        reg_alpha=5, reg_lambda=10, scale_pos_weight=1, silent=True,
        subsample=0.5, subsample_for_bin=200, subsample_freq=1)


best_parameter = {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.65, 'learning_rate': 0.1, 'max_depth': 11, 'min_child_samples': 5, 'min_child_weight': 1, 'min_split_gain': 0.5, 'n_estimators': 500, 'n_jobs': -1, 'num_leaves': 31, 'objective': 'binary', 'random_state': None, 'reg_alpha': 5, 'reg_lambda': 10, 'silent': True, 'subsa

In [6]:
import lightgbm as lgb  
from lightgbm import LGBMClassifier 
from time import time

params = {'boosting_type': 'gbdt',
          'max_depth' : -1,
          'objective': 'binary', 
          'nthread': 5, # Updated from nthread
          'num_leaves': 64, 
          'learning_rate': 0.05, 
          'max_bin': 512, 
          'subsample_for_bin': 200,
          'subsample': 1, 
          'subsample_freq': 1, 
          'colsample_bytree': 0.8, 
          'reg_alpha': 5, 
          'reg_lambda': 10,
          'min_split_gain': 0.5, 
          'min_child_weight': 1, 
          'min_child_samples': 5, 
          'scale_pos_weight': 1,
          'num_class' : 1,
          'metric' : 'binary_error',
          'cat_smooth' : 10,
          'objective' : 'binary',
          'metric' : 'auc'}

# Set the parameters by cross-validation
tuned_parameters = {
    #'learning_rate': [2, 1.5, 1, 0.5, 0.25, 0.1, 0.05, 0.01, 0.005, 0.001],
    'learning_rate': [0.05],
    'n_estimators': [500],
    "max_depth": [3, 5, 8, 9, 11, 12],
    #'num_leaves': range(30, 100, 30),
    'boosting_type' : ['gbdt'],
    #'random_state' : [501], # Updated from 'seed'
    'colsample_bytree' : [0.65],
    'subsample' : [0.5]
    #'reg_alpha' : [1, 1.2],
    #'reg_lambda' : [1, 1.2, 1.4],
    }

# Number of random trials
NUM_TRIALS = 1

# We will use a Support Vector Classifier with "rbf" kernel
#lgbm = LGBMClassifier()
lgbm = LGBMClassifier(boosting_type= 'gbdt',  
          n_jobs = -1, # Updated from 'nthread'
          silent = True,
          max_depth = params['max_depth'],
          max_bin = params['max_bin'], 
          subsample_for_bin = params['subsample_for_bin'],
          subsample = params['subsample'], 
          subsample_freq = params['subsample_freq'], 
          min_split_gain = params['min_split_gain'], 
          min_child_weight = params['min_child_weight'], 
          min_child_samples = params['min_child_samples'], 
          scale_pos_weight = params['scale_pos_weight'],
          cat_smooth = params['cat_smooth'],
          objective = params['objective'],
          reg_alpha = params['reg_alpha'],
          reg_lambda = params['reg_lambda'],
          metric = params['metric'])

start = time()
(max_score, lgbm_best_estimator) = CrossValidationGridSearchNested(X_val, Y_val.ravel(), NUM_TRIALS, 10, lgbm, tuned_parameters, 'roc_auc')
lgbm_best_parameter = lgbm_best_estimator.get_params()

print("CrossValidationGridSearchNested of LightGradientBoostingClassifier wih NUM_TRIALS = %2.0d took %.2f seconds."%(NUM_TRIALS, (time() - start)))
print(f'\nmax_score = {max_score}\n')
print(f'\nbest_estimator = {lgbm_best_estimator}\n')
print(f'\nbest_parameter = {lgbm_best_parameter}\n')

> progress = 100.0%
CrossValidationGridSearchNested of LightGradientBoostingClassifier wih NUM_TRIALS =  1 took 96357.52 seconds.

max_score = 0.9878247075153679


best_estimator = LGBMClassifier(boosting_type='gbdt', cat_smooth=10, class_weight=None,
        colsample_bytree=0.65, learning_rate=0.05, max_bin=512,
        max_depth=8, metric='auc', min_child_samples=5, min_child_weight=1,
        min_split_gain=0.5, n_estimators=500, n_jobs=-1, num_leaves=31,
        objective='binary', random_state=None, reg_alpha=5, reg_lambda=10,
        scale_pos_weight=1, silent=True, subsample=0.5,
        subsample_for_bin=200, subsample_freq=1)


best_parameter = {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.65, 'learning_rate': 0.05, 'max_depth': 8, 'min_child_samples': 5, 'min_child_weight': 1, 'min_split_gain': 0.5, 'n_estimators': 500, 'n_jobs': -1, 'num_leaves': 31, 'objective': 'binary', 'random_state': None, 'reg_alpha': 5, 'reg_lambda': 10, 'silent': True, 'subsa

In [7]:
# Set the parameters by cross-validation
tuned_parameters = {
    #'learning_rate': [2, 1.5, 1, 0.5, 0.25, 0.1, 0.05, 0.01, 0.005, 0.001],
    'learning_rate': [0.05],
    'n_estimators': [500],
    "max_depth": [8],
    'boosting_type' : ['gbdt'],
    #'random_state' : [501], # Updated from 'seed'
    'colsample_bytree' : [0.1, 0.65, 0.8, 1],
    'subsample' : [0.5]
    #'reg_alpha' : [1, 1.2],
    #'reg_lambda' : [1, 1.2, 1.4],
    }

# Number of random trials
NUM_TRIALS = 1

# We will use a Support Vector Classifier with "rbf" kernel
#lgbm = LGBMClassifier()
lgbm = LGBMClassifier(boosting_type= 'gbdt',  
          n_jobs = -1, # Updated from 'nthread'
          silent = True,
          max_depth = params['max_depth'],
          max_bin = params['max_bin'], 
          subsample_for_bin = params['subsample_for_bin'],
          subsample = params['subsample'], 
          subsample_freq = params['subsample_freq'], 
          min_split_gain = params['min_split_gain'], 
          min_child_weight = params['min_child_weight'], 
          min_child_samples = params['min_child_samples'], 
          scale_pos_weight = params['scale_pos_weight'],
          cat_smooth = params['cat_smooth'],
          objective = params['objective'],
          reg_alpha = params['reg_alpha'],
          reg_lambda = params['reg_lambda'],
          metric = params['metric'])

start = time()
(max_score, lgbm_best_estimator) = CrossValidationGridSearchNested(X_val, Y_val.ravel(), NUM_TRIALS, 10, lgbm, tuned_parameters, 'roc_auc')
lgbm_best_parameter = lgbm_best_estimator.get_params()

print("CrossValidationGridSearchNested of LightGradientBoostingClassifier wih NUM_TRIALS = %2.0d took %.2f seconds."%(NUM_TRIALS, (time() - start)))
print(f'\nmax_score = {max_score}\n')
print(f'\nbest_estimator = {lgbm_best_estimator}\n')
print(f'\nbest_parameter = {lgbm_best_parameter}\n')

> progress = 100.0%
CrossValidationGridSearchNested of LightGradientBoostingClassifier wih NUM_TRIALS =  1 took 61402.38 seconds.

max_score = 0.9878247075153679


best_estimator = LGBMClassifier(boosting_type='gbdt', cat_smooth=10, class_weight=None,
        colsample_bytree=0.65, learning_rate=0.05, max_bin=512,
        max_depth=8, metric='auc', min_child_samples=5, min_child_weight=1,
        min_split_gain=0.5, n_estimators=500, n_jobs=-1, num_leaves=31,
        objective='binary', random_state=None, reg_alpha=5, reg_lambda=10,
        scale_pos_weight=1, silent=True, subsample=0.5,
        subsample_for_bin=200, subsample_freq=1)


best_parameter = {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.65, 'learning_rate': 0.05, 'max_depth': 8, 'min_child_samples': 5, 'min_child_weight': 1, 'min_split_gain': 0.5, 'n_estimators': 500, 'n_jobs': -1, 'num_leaves': 31, 'objective': 'binary', 'random_state': None, 'reg_alpha': 5, 'reg_lambda': 10, 'silent': True, 'subsa

In [8]:
# Set the parameters by cross-validation
tuned_parameters = {
    #'learning_rate': [2, 1.5, 1, 0.5, 0.25, 0.1, 0.05, 0.01, 0.005, 0.001],
    'learning_rate': [0.05],
    'n_estimators': [500],
    "max_depth": [8],
    'boosting_type' : ['gbdt'],
    #'random_state' : [501], # Updated from 'seed'
    'colsample_bytree' : [0.65],
    'subsample' : [0.1, 0.5, 0.75, 1]
    #'reg_alpha' : [1, 1.2],
    #'reg_lambda' : [1, 1.2, 1.4],
    }

# Number of random trials
NUM_TRIALS = 1

# We will use a Support Vector Classifier with "rbf" kernel
#lgbm = LGBMClassifier()
lgbm = LGBMClassifier(boosting_type= 'gbdt',  
          n_jobs = -1, # Updated from 'nthread'
          silent = True,
          max_depth = params['max_depth'],
          max_bin = params['max_bin'], 
          subsample_for_bin = params['subsample_for_bin'],
          subsample = params['subsample'], 
          subsample_freq = params['subsample_freq'], 
          min_split_gain = params['min_split_gain'], 
          min_child_weight = params['min_child_weight'], 
          min_child_samples = params['min_child_samples'], 
          scale_pos_weight = params['scale_pos_weight'],
          cat_smooth = params['cat_smooth'],
          objective = params['objective'],
          reg_alpha = params['reg_alpha'],
          reg_lambda = params['reg_lambda'],
          metric = params['metric'])

start = time()
(max_score, lgbm_best_estimator) = CrossValidationGridSearchNested(X_val, Y_val.ravel(), NUM_TRIALS, 10, lgbm, tuned_parameters, 'roc_auc')
lgbm_best_parameter = lgbm_best_estimator.get_params()

print("CrossValidationGridSearchNested of LightGradientBoostingClassifier wih NUM_TRIALS = %2.0d took %.2f seconds."%(NUM_TRIALS, (time() - start)))
print(f'\nmax_score = {max_score}\n')
print(f'\nbest_estimator = {lgbm_best_estimator}\n')
print(f'\nbest_parameter = {lgbm_best_parameter}\n')

> progress = 100.0%
CrossValidationGridSearchNested of LightGradientBoostingClassifier wih NUM_TRIALS =  1 took 55809.89 seconds.

max_score = 0.9878247075153679


best_estimator = LGBMClassifier(boosting_type='gbdt', cat_smooth=10, class_weight=None,
        colsample_bytree=0.65, learning_rate=0.05, max_bin=512,
        max_depth=8, metric='auc', min_child_samples=5, min_child_weight=1,
        min_split_gain=0.5, n_estimators=500, n_jobs=-1, num_leaves=31,
        objective='binary', random_state=None, reg_alpha=5, reg_lambda=10,
        scale_pos_weight=1, silent=True, subsample=0.5,
        subsample_for_bin=200, subsample_freq=1)


best_parameter = {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.65, 'learning_rate': 0.05, 'max_depth': 8, 'min_child_samples': 5, 'min_child_weight': 1, 'min_split_gain': 0.5, 'n_estimators': 500, 'n_jobs': -1, 'num_leaves': 31, 'objective': 'binary', 'random_state': None, 'reg_alpha': 5, 'reg_lambda': 10, 'silent': True, 'subsa

In [6]:
import lightgbm as lgb  
from lightgbm import LGBMClassifier 
from time import time


params = {'boosting_type': 'gbdt',
          'max_depth' : -1,
          'objective': 'binary', 
          'nthread': 5, # Updated from nthread
          'num_leaves': 64, 
          'learning_rate': 0.05, 
          'max_bin': 512, 
          'subsample_for_bin': 200,
          'subsample': 1, 
          'subsample_freq': 1, 
          'colsample_bytree': 0.8, 
          'reg_alpha': 5, 
          'reg_lambda': 10,
          'min_split_gain': 0.5, 
          'min_child_weight': 1, 
          'min_child_samples': 5, 
          'scale_pos_weight': 1,
          'num_class' : 1,
          'metric' : 'binary_error',
          'cat_smooth' : 10,
          'objective' : 'binary',
          'metric' : 'auc'}


# Set the parameters by cross-validation
tuned_parameters = {
    'learning_rate': [2, 1.5, 1, 0.5, 0.25, 0.1, 0.05, 0.01, 0.005, 0.001],
    #'learning_rate': [0.05],
    'n_estimators': [500],
    "max_depth": [8],
    'boosting_type' : ['gbdt'],
    #'random_state' : [501], # Updated from 'seed'
    'colsample_bytree' : [0.65],
    'subsample' : [0.5],
    'reg_alpha' : [1e-5, 1e-2, 0.1, 1, 100, 1000]
    #'reg_lambda' : [1, 1.2, 1.4],
    }

# Number of random trials
NUM_TRIALS = 1

# We will use a Support Vector Classifier with "rbf" kernel
#lgbm = LGBMClassifier()
lgbm = LGBMClassifier(boosting_type= 'gbdt',  
          n_jobs = -1, # Updated from 'nthread'
          silent = True,
          max_depth = params['max_depth'],
          max_bin = params['max_bin'], 
          subsample_for_bin = params['subsample_for_bin'],
          subsample = params['subsample'], 
          subsample_freq = params['subsample_freq'], 
          min_split_gain = params['min_split_gain'], 
          min_child_weight = params['min_child_weight'], 
          min_child_samples = params['min_child_samples'], 
          scale_pos_weight = params['scale_pos_weight'],
          cat_smooth = params['cat_smooth'],
          objective = params['objective'],
          reg_alpha = params['reg_alpha'],
          reg_lambda = params['reg_lambda'],
          metric = params['metric'])

start = time()
(max_score, lgbm_best_estimator) = CrossValidationGridSearchNested(X_val, Y_val.ravel(), NUM_TRIALS, 10, lgbm, tuned_parameters, 'roc_auc')
lgbm_best_parameter = lgbm_best_estimator.get_params()

print("CrossValidationGridSearchNested of LightGradientBoostingClassifier wih NUM_TRIALS = %2.0d took %.2f seconds."%(NUM_TRIALS, (time() - start)))
print(f'\nmax_score = {max_score}\n')
print(f'\nbest_estimator = {lgbm_best_estimator}\n')
print(f'\nbest_parameter = {lgbm_best_parameter}\n')

> progress = 100.0%
CrossValidationGridSearchNested of LightGradientBoostingClassifier wih NUM_TRIALS =  1 took 115888.47 seconds.

max_score = 0.9919161874545577


best_estimator = LGBMClassifier(boosting_type='gbdt', cat_smooth=10, class_weight=None,
        colsample_bytree=0.65, learning_rate=0.1, max_bin=512, max_depth=8,
        metric='auc', min_child_samples=5, min_child_weight=1,
        min_split_gain=0.5, n_estimators=500, n_jobs=-1, num_leaves=31,
        objective='binary', random_state=None, reg_alpha=1e-05,
        reg_lambda=10, scale_pos_weight=1, silent=True, subsample=0.5,
        subsample_for_bin=200, subsample_freq=1)


best_parameter = {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.65, 'learning_rate': 0.1, 'max_depth': 8, 'min_child_samples': 5, 'min_child_weight': 1, 'min_split_gain': 0.5, 'n_estimators': 500, 'n_jobs': -1, 'num_leaves': 31, 'objective': 'binary', 'random_state': None, 'reg_alpha': 1e-05, 'reg_lambda': 10, 'silent': True,

In [7]:
import lightgbm as lgb  
from lightgbm import LGBMClassifier 
from time import time


params = {'boosting_type': 'gbdt',
          'max_depth' : -1,
          'objective': 'binary', 
          'nthread': 5, # Updated from nthread
          'num_leaves': 64, 
          'learning_rate': 0.05, 
          'max_bin': 512, 
          'subsample_for_bin': 200,
          'subsample': 1, 
          'subsample_freq': 1, 
          'colsample_bytree': 0.8, 
          'reg_alpha': 5, 
          'reg_lambda': 10,
          'min_split_gain': 0.5, 
          'min_child_weight': 1, 
          'min_child_samples': 5, 
          'scale_pos_weight': 1,
          'num_class' : 1,
          'metric' : 'binary_error',
          'cat_smooth' : 10,
          'objective' : 'binary',
          'metric' : 'auc'}


# Set the parameters by cross-validation
tuned_parameters = {
    'learning_rate': [1, 0.5, 0.25, 0.1, 0.05, 0.01],
    #'learning_rate': [0.05],
    'n_estimators': [500],
    "max_depth": [8],
    'boosting_type' : ['gbdt'],
    #'random_state' : [501], # Updated from 'seed'
    'colsample_bytree' : [0.65],
    'subsample' : [0.5],
    'reg_alpha' : [1e-7, 1e-6, 1e-5, 1e-2, 0.1, 1, 100, 1000]
    #'reg_lambda' : [1, 1.2, 1.4],
    }

# Number of random trials
NUM_TRIALS = 1

# We will use a Support Vector Classifier with "rbf" kernel
#lgbm = LGBMClassifier()
lgbm = LGBMClassifier(boosting_type= 'gbdt',  
          n_jobs = -1, # Updated from 'nthread'
          silent = True,
          max_depth = params['max_depth'],
          max_bin = params['max_bin'], 
          subsample_for_bin = params['subsample_for_bin'],
          subsample = params['subsample'], 
          subsample_freq = params['subsample_freq'], 
          min_split_gain = params['min_split_gain'], 
          min_child_weight = params['min_child_weight'], 
          min_child_samples = params['min_child_samples'], 
          scale_pos_weight = params['scale_pos_weight'],
          cat_smooth = params['cat_smooth'],
          objective = params['objective'],
          reg_alpha = params['reg_alpha'],
          reg_lambda = params['reg_lambda'],
          metric = params['metric'])

start = time()
(max_score, lgbm_best_estimator) = CrossValidationGridSearchNested(X_val, Y_val.ravel(), NUM_TRIALS, 10, lgbm, tuned_parameters, 'roc_auc')
lgbm_best_parameter = lgbm_best_estimator.get_params()

print("CrossValidationGridSearchNested of LightGradientBoostingClassifier wih NUM_TRIALS = %2.0d took %.2f seconds."%(NUM_TRIALS, (time() - start)))
print(f'\nmax_score = {max_score}\n')
print(f'\nbest_estimator = {lgbm_best_estimator}\n')
print(f'\nbest_parameter = {lgbm_best_parameter}\n')

> progress = 100.0%
CrossValidationGridSearchNested of LightGradientBoostingClassifier wih NUM_TRIALS =  1 took 156517.27 seconds.

max_score = 0.9919161874545577


best_estimator = LGBMClassifier(boosting_type='gbdt', cat_smooth=10, class_weight=None,
        colsample_bytree=0.65, learning_rate=0.1, max_bin=512, max_depth=8,
        metric='auc', min_child_samples=5, min_child_weight=1,
        min_split_gain=0.5, n_estimators=500, n_jobs=-1, num_leaves=31,
        objective='binary', random_state=None, reg_alpha=1e-05,
        reg_lambda=10, scale_pos_weight=1, silent=True, subsample=0.5,
        subsample_for_bin=200, subsample_freq=1)


best_parameter = {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.65, 'learning_rate': 0.1, 'max_depth': 8, 'min_child_samples': 5, 'min_child_weight': 1, 'min_split_gain': 0.5, 'n_estimators': 500, 'n_jobs': -1, 'num_leaves': 31, 'objective': 'binary', 'random_state': None, 'reg_alpha': 1e-05, 'reg_lambda': 10, 'silent': True,

In [4]:
import lightgbm as lgb  
from lightgbm import LGBMClassifier 
from time import time


params = {'boosting_type': 'gbdt',
          'max_depth' : -1,
          'objective': 'binary', 
          'nthread': 5, # Updated from nthread
          'num_leaves': 64, 
          'learning_rate': 0.05, 
          'max_bin': 512, 
          'subsample_for_bin': 200,
          'subsample': 1, 
          'subsample_freq': 1, 
          'colsample_bytree': 0.8, 
          'reg_alpha': 5, 
          'reg_lambda': 10,
          'min_split_gain': 0.5, 
          'min_child_weight': 1, 
          'min_child_samples': 5, 
          'scale_pos_weight': 1,
          'num_class' : 1,
          'metric' : 'binary_error',
          'cat_smooth' : 10,
          'objective' : 'binary',
          'metric' : 'auc'}


# Set the parameters by cross-validation
tuned_parameters = {
    'learning_rate': [0.1],
    #'learning_rate': [0.05],
    'n_estimators': [500],
    "max_depth": [8],
    'boosting_type' : ['gbdt'],
    #'random_state' : [501], # Updated from 'seed'
    'colsample_bytree' : [0.65],
    'subsample' : [0.5],
    'reg_alpha' : [1e-7, 1e-6, 1e-5, 1e-2, 0.1, 1, 100, 1000],
    'reg_lambda' : [1e-7, 1e-6, 1e-5, 1e-2, 0.1, 1, 100, 1000],
    }

# Number of random trials
NUM_TRIALS = 1

# We will use a Support Vector Classifier with "rbf" kernel
#lgbm = LGBMClassifier()
lgbm = LGBMClassifier(boosting_type= 'gbdt',  
          n_jobs = -1, # Updated from 'nthread'
          silent = True,
          max_depth = params['max_depth'],
          max_bin = params['max_bin'], 
          subsample_for_bin = params['subsample_for_bin'],
          subsample = params['subsample'], 
          subsample_freq = params['subsample_freq'], 
          min_split_gain = params['min_split_gain'], 
          min_child_weight = params['min_child_weight'], 
          min_child_samples = params['min_child_samples'], 
          scale_pos_weight = params['scale_pos_weight'],
          cat_smooth = params['cat_smooth'],
          objective = params['objective'],
          reg_alpha = params['reg_alpha'],
          reg_lambda = params['reg_lambda'],
          metric = params['metric'])

start = time()
(max_score, lgbm_best_estimator) = CrossValidationGridSearchNested(X_val, Y_val.ravel(), NUM_TRIALS, 10, lgbm, tuned_parameters, 'roc_auc')
lgbm_best_parameter = lgbm_best_estimator.get_params()

print("CrossValidationGridSearchNested of LightGradientBoostingClassifier wih NUM_TRIALS = %2.0d took %.2f seconds."%(NUM_TRIALS, (time() - start)))
print(f'\nmax_score = {max_score}\n')
print(f'\nbest_estimator = {lgbm_best_estimator}\n')
print(f'\nbest_parameter = {lgbm_best_parameter}\n')

> progress = 100.0%
CrossValidationGridSearchNested of LightGradientBoostingClassifier wih NUM_TRIALS =  1 took 144823.09 seconds.

max_score = 0.9902967810165906


best_estimator = LGBMClassifier(boosting_type='gbdt', cat_smooth=10, class_weight=None,
        colsample_bytree=0.65, learning_rate=0.1, max_bin=512, max_depth=8,
        metric='auc', min_child_samples=5, min_child_weight=1,
        min_split_gain=0.5, n_estimators=500, n_jobs=-1, num_leaves=31,
        objective='binary', random_state=None, reg_alpha=0.01,
        reg_lambda=1, scale_pos_weight=1, silent=True, subsample=0.5,
        subsample_for_bin=200, subsample_freq=1)


best_parameter = {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.65, 'learning_rate': 0.1, 'max_depth': 8, 'min_child_samples': 5, 'min_child_weight': 1, 'min_split_gain': 0.5, 'n_estimators': 500, 'n_jobs': -1, 'num_leaves': 31, 'objective': 'binary', 'random_state': None, 'reg_alpha': 0.01, 'reg_lambda': 1, 'silent': True, 'su